In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

NLP From Scratch: Translation with a Sequence to Sequence Network and Attention
===============================================================================

**Author**: [Sean Robertson](https://github.com/spro)

This is the third and final tutorial on doing \"NLP From Scratch\",
where we write our own classes and functions to preprocess the data to
do our NLP modeling tasks. We hope after you complete this tutorial that
you\'ll proceed to learn how [torchtext]{.title-ref} can handle much of
this preprocessing for you in the three tutorials immediately following
this one.

In this project we will be teaching a neural network to translate from
French to English.

``` {.sourceCode .sh}
[KEY: > input, = target, < output]

> il est en train de peindre un tableau .
= he is painting a picture .
< he is painting a picture .

> pourquoi ne pas essayer ce vin delicieux ?
= why not try that delicious wine ?
< why not try that delicious wine ?

> elle n est pas poete mais romanciere .
= she is not a poet but a novelist .
< she not not a poet but a novelist .

> vous etes trop maigre .
= you re too skinny .
< you re all alone .
```

\... to varying degrees of success.

This is made possible by the simple but powerful idea of the [sequence
to sequence network](https://arxiv.org/abs/1409.3215), in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

To improve upon this model we\'ll use an [attention
mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-   <https://pytorch.org/> For installation instructions
-   `/beginner/deep_learning_60min_blitz`{.interpreted-text role="doc"}
    to get started with PyTorch in general
-   `/beginner/pytorch_with_examples`{.interpreted-text role="doc"} for
    a wide and deep overview
-   `/beginner/former_torchies_tutorial`{.interpreted-text role="doc"}
    if you are former Lua Torch user

It would also be useful to know about Sequence to Sequence networks and
how they work:

-   [Learning Phrase Representations using RNN Encoder-Decoder for
    Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
-   [Sequence to Sequence Learning with Neural
    Networks](https://arxiv.org/abs/1409.3215)
-   [Neural Machine Translation by Jointly Learning to Align and
    Translate](https://arxiv.org/abs/1409.0473)
-   [A Neural Conversational Model](https://arxiv.org/abs/1506.05869)

You will also find the previous tutorials on
`/intermediate/char_rnn_classification_tutorial`{.interpreted-text
role="doc"} and
`/intermediate/char_rnn_generation_tutorial`{.interpreted-text
role="doc"} helpful as those concepts are very similar to the Encoder
and Decoder models, respectively.

**Requirements**


In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import copy

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

[This question on Open Data Stack
Exchange](https://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages)
pointed me to the open translation site <https://tatoeba.org/> which has
downloads available at <https://tatoeba.org/eng/downloads> - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: <https://www.manythings.org/anki/>

The English to French pairs are too big to include in the repository, so
download to `data/eng-fra.txt` before continuing. The file is a tab
separated list of translation pairs:

``` {.sourceCode .sh}
I am cold.    J'ai froid.
```

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>Download the data from<a href="https://download.pytorch.org/tutorial/data.zip">here</a>and extract it to the current directory.</p>
</div>


Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/word-encoding.png)


We\'ll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called `Lang` which has word → index (`word2index`) and index → word
(`index2word`) dictionaries, as well as a count of each word
`word2count` which will be used to replace rare words later.


In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.


In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the `reverse`
flag to reverse the pairs.


In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we\'ll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we\'re filtering to sentences that translate to
the form \"I am\" or \"He is\" etc. (accounting for apostrophes replaced
earlier).


In [81]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH # and \
        # p[1].startswith(eng_prefixes) #RECOMMENT THIS LATER


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-   Read text file and split into lines, split lines into pairs
-   Normalize text, filter by length and content
-   Make word lists from sentences in pairs


In [46]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'cat', True)
print(random.choice(pairs))

Reading lines...
Read 1375 sentence pairs
Trimmed to 1131 sentence pairs
Counting words...
Counted words:
cat 1496
eng 1220
['tom pot parlar amb mary si vol', 'tom may talk to mary if he wants to']


The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215), or
seq2seq network, or [Encoder Decoder
network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting
of two RNNs called the encoder and decoder. The encoder reads an input
sequence and outputs a single vector, and the decoder reads that vector
to produce an output sequence.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence `Je ne suis pas le chat noir` →
`I am not the black cat`. Most of the words in the input sentence have a
direct translation in the output sentence, but are in slightly different
orders, e.g. `chat noir` and `black cat`. Because of the `ne/pas`
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the \"meaning\" of the input sequence into a single
vector --- a single point in some N dimensional space of sentences.


The Encoder
===========

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/encoder-network.png)


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

The Decoder
===========

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.


Simple Decoder
==============

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string `<SOS>`
token, and the first hidden state is the context vector (the encoder\'s
last hidden state).

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/decoder-network.png)


In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

I encourage you to train and observe the results of this model, but to
save space we\'ll be going straight for the gold and introducing the
Attention Mechanism.


Attention Decoder
=================

If only the context vector is passed between the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to \"focus\" on a different part of
the encoder\'s outputs for every step of the decoder\'s own outputs.
First we calculate a set of *attention weights*. These will be
multiplied by the encoder output vectors to create a weighted
combination. The result (called `attn_applied` in the code) should
contain information about that specific part of the input sequence, and
thus help the decoder choose the right output words.

![](https://i.imgur.com/1152PYf.png)

Calculating the attention weights is done with another feed-forward
layer `attn`, using the decoder\'s input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/attention-decoder-network.png)

Bahdanau attention, also known as additive attention, is a commonly used
attention mechanism in sequence-to-sequence models, particularly in
neural machine translation tasks. It was introduced by Bahdanau et al.
in their paper titled [Neural Machine Translation by Jointly Learning to
Align and Translate](https://arxiv.org/pdf/1409.0473.pdf). This
attention mechanism employs a learned alignment model to compute
attention scores between the encoder and decoder hidden states. It
utilizes a feed-forward neural network to calculate alignment scores.

However, there are alternative attention mechanisms available, such as
Luong attention, which computes attention scores by taking the dot
product between the decoder hidden state and the encoder hidden states.
It does not involve the non-linear transformation used in Bahdanau
attention.

In this tutorial, we will be using Bahdanau attention. However, it would
be a valuable exercise to explore modifying the attention mechanism to
use Luong attention.


In [9]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [10]:
class EncoderMetaModel(nn.Module):
    def __init__(self, encoders_dict, decoders_dict):
        super(EncoderMetaModel, self).__init__()
        self.encoders_dict = nn.ModuleDict(encoders_dict)
        self.decoders_dict = nn.ModuleDict(decoders_dict)
        self.encoder_weights = nn.Parameter(torch.ones(len(encoders_dict)) / len(encoders_dict))
        self.decoder_weights = nn.Parameter(torch.ones(len(decoders_dict)) / len(decoders_dict))
    
    def forward(self, x):
        encoder_combined_output = 0
        w_encoder = []
        # w_encoder.append({key: value for key, value in copy.deepcopy(encoder_weights).items() if 'embedding' not in key})
        # w_decoder.append({key: value for key, value in copy.deepcopy(decoder_weights).items() if 'embedding' not in key and 'out' not in key})
        for i, encoder in enumerate(self.encoders_dict.values()):
            encoder_combined_output += self.encoder_weights[i] * encoder(x)
        
        decoder_combined_output = 0
        for i, decoder in enumerate(self.decoders_dict.values()):
            decoder_combined_output += self.decoder_weights[i] * decoder(encoder_combined_output)
        
        return decoder_combined_output

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>There are other forms of attention that work around the lengthlimitation by using a relative position approach. Read about "localattention" in <a href="https://arxiv.org/abs/1508.04025">Effective Approaches to Attention-based Neural MachineTranslation</a>.</p>
</div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.


In [11]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size, language='spa'):
    input_lang, output_lang, pairs = prepareData('eng', language, True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    indices_tensor = torch.arange(n)

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device),
                               indices_tensor.to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader, pairs

Training the Model
==================

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the `<SOS>` token as its first input, and the last hidden state of the
encoder as its first hidden state.

\"Teacher forcing\" is the concept of using the real target outputs as
each next input, instead of using the decoder\'s guess as the next
input. Using teacher forcing causes it to converge faster but [when the
trained network is exploited, it may exhibit
instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation
-intuitively it has learned to represent the output grammar and can
\"pick up\" the meaning once the teacher tells it the first few words,
but it has not properly learned how to create the sentence from the
translation in the first place.

Because of the freedom PyTorch\'s autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
`teacher_forcing_ratio` up to use more of it.


In [12]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor, _ = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.


In [13]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
class ClientUpdate(object):
    def __init__(self, train_dataloader, learning_rate, epochs, sch_flag):
        self.train_loader = train_dataloader
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.sch_flag = sch_flag

    def train(self, encoder, decoder):

        criterion = nn.CrossEntropyLoss()
        # optimizer = torch.optim.SGD(model.parameters(), lr=self.learning_rate, momentum=0.95, weight_decay = 5e-4)
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=self.learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=self.learning_rate)
        # if self.sch_flag == True:
        #    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5)
        # my_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
        e_loss = []
        for epoch in range(1, self.epochs + 1):
            loss = train_epoch(self.train_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            e_loss.append(loss)

            # print_loss_total += loss
            # plot_loss_total += loss

            # if epoch % print_every == 0:
            #     print_loss_avg = print_loss_total / print_every
            #     print_loss_total = 0
            #     print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
            #                                 epoch, epoch / n_epochs * 100, print_loss_avg))

            # if epoch % plot_every == 0:
            #     plot_loss_avg = plot_loss_total / plot_every
            #     plot_losses.append(plot_loss_avg)
            #     plot_loss_total = 0

            total_loss = sum(e_loss) / len(e_loss)

        return encoder.state_dict(), decoder.state_dict(), total_loss, torch.tensor(len(self.train_loader.sampler))

The whole training process looks like this:

-   Start a timer
-   Initialize optimizers and criterion
-   Create set of training pairs
-   Start empty losses array for plotting

Then we call `train` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.


In [15]:
from torchtext.data.metrics import bleu_score
def evaluateBleu(encoder, decoder, input_lang, output_lang, pairs, n=10, verbose=False):
    references = []
    candidates = []

    numbers = random.sample(range(len(pairs)), n)

    for num in numbers:
        pair = pairs[num]
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words).split(' ')
        if verbose:
            print('>', pair[0])
            print('=', pair[1])
            print('<', ' '.join(output_sentence))
            print('')

        # Store the reference and candidate sentences for BLEU calculation
        references.append([pair[1].split(' ')])
        candidates.append(output_sentence)

    # Calculate the BLEU score
    score = bleu_score(candidates, references)
    return score

In [16]:
import csv
def train(train_dataloader, encoder, decoder, n_epochs, input, output, pairs, test_pairs=None, filename=None, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    best_bleu = 0
    best_test_bleu = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        bleu = evaluateBleu(encoder, decoder, input, output, pairs, n=min(200,len(pairs)))
        if test_pairs is not None:
            test_bleu = evaluateBleu(encoder, decoder, input, output, test_pairs, n=min(200,len(test_pairs)))
        else:
            test_bleu = 0

        if best_bleu < bleu:
            best_bleu = bleu
        
        if best_test_bleu < test_bleu:
            best_test_bleu = test_bleu

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f'Epoch {epoch} >> Loss: {print_loss_avg}, Bleu: {bleu}, Test Bleu: {test_bleu}, Best:({best_bleu}/{best_test_bleu})')

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


        if filename is not None:
            with open(filename, 'a') as f:
                # create the csv writer
                writer = csv.writer(f)

                # write a row to the csv file
                writer.writerow([epoch, loss, bleu, best_bleu, test_bleu, best_test_bleu])

    showPlot(plot_losses)

In [17]:
import csv
def train_frozen(train_dataloader, encoder, decoder, e_dict, d_dict, n_epochs, input, output, pairs, test_pairs=None, filename=None, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    best_bleu = 0
    best_test_bleu = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        e_og = encoder.state_dict()
        d_og = decoder.state_dict()
        e_og.update(e_dict)
        d_og.update(d_dict)
        encoder.load_state_dict(e_og)
        decoder.load_state_dict(d_og)

        is_FL = 'true'
        print_loss_total += loss
        plot_loss_total += loss

        bleu = evaluateBleu(encoder, decoder, input, output, pairs, n=min(200,len(pairs)))
        if test_pairs is not None:
            test_bleu = evaluateBleu(encoder, decoder, input, output, test_pairs, n=min(200,len(test_pairs)))
        else:
            test_bleu = 0

        if best_bleu < bleu:
            best_bleu = bleu
        
        if best_test_bleu < test_bleu:
            best_test_bleu = test_bleu

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f'Epoch {epoch} >> Loss: {print_loss_avg}, Bleu: {bleu}, Test Bleu: {test_bleu}, Best:({best_bleu}/{best_test_bleu})')

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


        if filename is not None:
            with open(filename, 'a') as f:
                # create the csv writer
                writer = csv.writer(f)

                # write a row to the csv file
                writer.writerow([epoch, loss, bleu, best_bleu, test_bleu, best_test_bleu])

    showPlot(plot_losses)

In [18]:
hidden_size = 128
batch_size = 32

In [47]:
def get_pair_index(dataloader):
    shuffled_indices = []
    for _, _, indices in dataloader:
        shuffled_indices.extend(indices.tolist())
    return shuffled_indices
from torch.utils.data import DataLoader, SubsetRandomSampler

def split_dataloader(dataloader, train_fraction=0.75):
    # Total number of samples in the dataloader
    dataset_size = len(dataloader.dataset)
    
    # Indices of all elements in the dataset
    indices = list(range(dataset_size))
    
    # Shuffle indices
    np.random.shuffle(indices)
    
    # Determine the split point
    split = int(np.floor(train_fraction * dataset_size))
    
    # Create training and test samplers
    train_indices, test_indices = indices[:split], indices[split:]
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    
    # Create training and test dataloaders
    train_loader = DataLoader(dataloader.dataset, batch_size=dataloader.batch_size, 
                              sampler=train_sampler, drop_last=True)
    test_loader = DataLoader(dataloader.dataset, batch_size=dataloader.batch_size, 
                             sampler=test_sampler, drop_last=True)
    
    return train_loader, test_loader, split

import torch
from torch.utils.data import DataLoader, Subset

def limited_data_loader(original_dataloader, num_samples, split, random=False):
    # Get the original dataset from the DataLoader
    dataset_size = len(original_dataloader.sampler.indices)
    dataset = original_dataloader.dataset
    
    # Ensure the original dataset is larger than the requested number of samples
    assert dataset_size >= num_samples, f"The original dataset has fewer samples than requested {dataset_size}"
    
    # Create a Subset of the original dataset limited to the first num_samples elements
    indices = original_dataloader.sampler.indices

    if random:
        # indices = np.random.choice(len(dataset), num_samples, replace=False)
        np.random.shuffle(indices)
    else:
        # indices = range(num_samples)
        print("no shuffle")

    # subset = Subset(dataset, indices)
    train_indices = indices[:num_samples]
    train_sampler = SubsetRandomSampler(train_indices)

    # Create a new DataLoader from this subset with the same parameters as the original DataLoader
    # new_dataloader = DataLoader(subset, batch_size=original_dataloader.batch_size, shuffle=False, num_workers=original_dataloader.num_workers)
    new_dataloader = DataLoader(original_dataloader.dataset, batch_size=original_dataloader.batch_size, 
                            sampler=train_sampler, drop_last=True)
    
    return new_dataloader


In [48]:
import os
def personalize(lang, rounds, encoder_weights=None, decoder_weights=None, sample=None, save=False, lang_info=None):
    if lang_info is None:
        input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language=lang)
    else:
        input_lang, output_lang, train_dataloader, pairs = lang_info

    encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

    is_FL = 'false'

    if encoder_weights is not None:
        e_og = encoder.state_dict()
        d_og = decoder.state_dict()
        e_og.update(encoder_weights)
        d_og.update(decoder_weights)
        encoder.load_state_dict(e_og)
        decoder.load_state_dict(d_og)
        is_FL = 'true'

    train_dataloader, test_dataloader, split = split_dataloader(train_dataloader)
  
    if sample is None:
        sample = 'full'
    else:
        train_dataloader = limited_data_loader(train_dataloader, sample, split)
    
    train_idx = get_pair_index(train_dataloader)
    test_idx = get_pair_index(test_dataloader)

    train_pairs = [pairs[i] for i in train_idx]
    test_pairs = [pairs[i] for i in test_idx]

    filename=None
    if save:
        num = 1
        filename = f"P|{lang}_{sample}-shot_FL-{is_FL}_epoch{rounds}||{num}.csv"
        while os.path.isfile(filename):
            print('Name is taken...trying again...')
            num += 1
            filename = f"P|{lang}_{sample}-shot_FL-{is_FL}_epoch{rounds}||{num}.csv"
    train(train_dataloader, encoder, decoder, rounds, print_every=25, plot_every=5, filename=filename, input=input_lang, output=output_lang, pairs=train_pairs, test_pairs=test_pairs)
    return encoder, decoder, test_pairs

In [49]:
import os
def personalize_frozen(lang, rounds, encoder_weights=None, decoder_weights=None, sample=None, save=False, lang_info=None):
    if lang_info is None:
        input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language=lang)
    else:
        input_lang, output_lang, train_dataloader, pairs = lang_info

    encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

    is_FL = 'false'

    if encoder_weights is not None:
        e_og = encoder.state_dict()
        d_og = decoder.state_dict()
        e_og.update(encoder_weights)
        d_og.update(decoder_weights)
        encoder.load_state_dict(e_og)
        decoder.load_state_dict(d_og)
        is_FL = 'true'

    train_dataloader, test_dataloader, split = split_dataloader(train_dataloader)
  
    if sample is None:
        sample = 'full'
    else:
        train_dataloader = limited_data_loader(train_dataloader, sample, split)
    
    train_idx = get_pair_index(train_dataloader)
    test_idx = get_pair_index(test_dataloader)

    train_pairs = [pairs[i] for i in train_idx]
    test_pairs = [pairs[i] for i in test_idx]

    filename=None
    if save:
        num = 1
        filename = f"P|{lang}_{sample}-shot_FL-{is_FL}_epoch{rounds}||{num}.csv"
        while os.path.isfile(filename):
            print('Name is taken...trying again...')
            num += 1
            filename = f"P|{lang}_{sample}-shot_FL-{is_FL}_epoch{rounds}||{num}.csv"
    train_frozen(train_dataloader, encoder, decoder, encoder_weights, decoder_weights, rounds, print_every=25, plot_every=5, filename=filename, input=input_lang, output=output_lang, pairs=train_pairs, test_pairs=test_pairs)
    return encoder, decoder, test_pairs

In [50]:
class MetaModel(nn.Module):
    def __init__(self, hidden_size, encoder_models, decoder_models, lang, update_rate=5, encoder=True, dropout_p=0.1):
        super(MetaModel, self).__init__()
        input_lang, output_lang, target_dataloader, pairs = get_dataloader(batch_size, language=lang)
        self.lang = lang
        self.encoder_models = encoder_models
        self.decoder_models = decoder_models
        self.hidden_size = hidden_size
        self.input_lang = input_lang
        self.output_lang = output_lang
        self.global_encoder_model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
        self.global_decoder_model = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
        self.pass_num = 0
        self.update_rate = update_rate
        self.num_models = len(encoder_models)
        self.proportions_encoder = nn.Parameter(torch.ones(self.num_models) / self.num_models)
        self.proportions_decoder = nn.Parameter(torch.ones(self.num_models) / self.num_models)
        self.encoder = encoder
        self.device = device
        self.encoder_optimizer = torch.optim.SGD([self.proportions_encoder], lr=0.1)
        self.decoder_optimizer = torch.optim.SGD([self.proportions_decoder], lr=0.1)
        self.target_dataloader = target_dataloader
        self.criterion = nn.NLLLoss()

    def train(self):
        encoder_weights = []
        decoder_weights = []

        # Reset Heads
        if self.pass_num % self.update_rate:
            self.global_encoder_model = EncoderRNN(self.input_lang.n_words, self.hidden_size).to(self.device)
            self.global_decoder_model = AttnDecoderRNN(self.hidden_size, self.output_lang.n_words).to(self.device)
        
        # Train head for target lang
        if (self.pass_num % self.update_rate) == 100:
            self.global_encoder_model, self.global_decoder_model, test_pairs = personalize(self.lang, 50, save=True, encoder_weights=self.global_encoder_model.state_dict(), decoder_weights=self.global_decoder_model.state_dict(), sample=None)
        self.pass_num += 1
            
        # Combine Decoder and Encoders
        # Make a list of encoder and decoder weights
        for i in range(len(self.encoder_models)):
            encoder_weights.append({key: value for key, value in copy.deepcopy(self.encoder_models[i].state_dict()).items() if 'embedding' not in key})
            decoder_weights.append({key: value for key, value in copy.deepcopy(self.decoder_models[i].state_dict()).items() if 'embedding' not in key and 'out' not in key})
        
        weights_avg_e = copy.deepcopy(encoder_weights[0])
        weights_avg_d = copy.deepcopy(decoder_weights[0])

        # Make state_dict empty with zeroes
        for k in weights_avg_e:
            weights_avg_e[k].zero_()
        for k in weights_avg_d:
            weights_avg_d[k].zero_()
            
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()
        # Add and combine weights of all input encoders/decoders according to proportion
        for k in weights_avg_e.keys():
            for i in range(0, len(encoder_weights)):
                weights_avg_e[k] += self.proportions_encoder[i] * encoder_weights[i][k]
        
        for k in weights_avg_d.keys():
            for i in range(0, len(decoder_weights)):
                weights_avg_d[k] += self.proportions_decoder[i] * decoder_weights[i][k]

        # Aggregate all encoders and decoders to update globals
        encoder_og = self.global_encoder_model.state_dict()
        encoder_og.update(weights_avg_e)
        self.global_encoder_model.load_state_dict(encoder_og)

        decoder_og = self.global_decoder_model.state_dict()
        decoder_og.update(weights_avg_d)
        self.global_decoder_model.load_state_dict(decoder_og)

        loss = self.train_meta_epoch(self.target_dataloader, self.global_encoder_model, self.global_decoder_model, self.encoder_optimizer, self.decoder_optimizer, self.criterion)
        
        return self.proportions_encoder, self.proportions_decoder, loss
    
    def train_meta_epoch(self, dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
        total_loss = 0
        for data in dataloader:
            input_tensor, target_tensor, _ = data

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)
            )
            loss.backward()

            # print(self.proportions_encoder.grad)

            encoder_optimizer.step()
            decoder_optimizer.step()

            total_loss += loss.item()

        return total_loss / len(dataloader)

In [51]:
def train_meta_epoch(self, dataloader, encoder_models, decoder_models, proportions_encoder, proportions_decoder, encoder_optimizer, decoder_optimizer, criterion, device='cuda'):
        total_loss = 0
        for data in dataloader:
            input_tensor, target_tensor, _ = data

            self.encoder_optimizer.zero_grad()
            self.decoder_optimizer.zero_grad()

            encoder_state_dict = combine_weights(encoder_models, proportions_encoder, device)
            decoder_state_dict = combine_weights(decoder_models, proportions_decoder, device)

            # Load the combined weights into a single encoder and decoder
            encoder.load_state_dict(encoder_state_dict)
            decoder.load_state_dict(decoder_state_dict)

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)
            )
            loss.backward()

            encoder_optimizer.step()
            decoder_optimizer.step()

            total_loss += loss.item()

        return total_loss / len(dataloader)

def combine_weights(models, proportions, device):
    combined_weights = copy.deepcopy(models[0].state_dict())
    for key in combined_weights.keys():
        combined_weights[key].zero_()
        for i, model in enumerate(models):
            model_weights = model.state_dict()
            combined_weights[key] += proportions[i] * model_weights[key].to(device)
    return combined_weights

# OPTUNA

In [105]:
import optuna
from functools import partial

def objective(trial, encoder_models, decoder_models, input_lang, output_lang, target_dataloader, target_pairs, test_pairs, sources, device):
    batch_size = 32
    hidden_size = 128
    
    STEP_SIZE = 1

    encoder_weights = []
    decoder_weights = []

    # This is what we fine-tune for
    proportions_encoder = torch.ones(len(encoder_models))
    proportions_decoder = torch.ones(len(encoder_models))

    # Reset Heads
    global_encoder_model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    global_decoder_model = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
    # global_encoder_model = global_model[0]
    # global_decoder_model = global_model[1]
        
    # Combine Decoder and Encoders
    # Make a list of encoder and decoder weights
    for i in range(len(encoder_models)):
        encoder_weights.append({key: value for key, value in copy.deepcopy(encoder_models[i].state_dict()).items() if 'embedding' not in key})
        decoder_weights.append({key: value for key, value in copy.deepcopy(decoder_models[i].state_dict()).items() if 'embedding' not in key and 'out' not in key})
    
    weights_avg_e = copy.deepcopy(encoder_weights[0])
    weights_avg_d = copy.deepcopy(decoder_weights[0])

    # Make state_dict empty with zeroes
    for k in weights_avg_e:
        weights_avg_e[k].zero_()
    for k in weights_avg_d:
        weights_avg_d[k].zero_()
    
    upper_limit_decoder = upper_limit_encoder = 100

    for i, source_lang in enumerate(sources[:-1]):
        proportions_encoder[i] = trial.suggest_int(f"{source_lang}_encoder", 0, upper_limit_encoder, step=STEP_SIZE)
        proportions_decoder[i] = trial.suggest_int(f"{source_lang}_decoder", 0, upper_limit_decoder, step=STEP_SIZE)
        upper_limit_encoder -= proportions_encoder[i]
        upper_limit_decoder -= proportions_decoder[i]
    proportions_encoder[-1] = 100 - upper_limit_encoder
    proportions_decoder[-1] = 100 - upper_limit_decoder
        
    # Add and combine weights of all input encoders/decoders according to proportion
    for k in weights_avg_e.keys():
        for i in range(0, len(encoder_weights)):
            weights_avg_e[k] += proportions_encoder[i] * encoder_weights[i][k] / 100
    
    for k in weights_avg_d.keys():
        for i in range(0, len(decoder_weights)):
            
            weights_avg_d[k] += proportions_decoder[i] * decoder_weights[i][k] / 100

    # Aggregate all encoders and decoders to update globals
    encoder_og = global_encoder_model.state_dict()
    encoder_og.update(weights_avg_e)
    global_encoder_model.load_state_dict(encoder_og)

    decoder_og = global_decoder_model.state_dict()
    decoder_og.update(weights_avg_d)
    global_decoder_model.load_state_dict(decoder_og)

    # Train head for target lang
    global_encoder_model, global_decoder_model, test_pairs = personalize('kir', 10, save=False, encoder_weights=global_encoder_model.state_dict(), decoder_weights=global_decoder_model.state_dict(), sample=None, lang_info=(input_lang, output_lang, target_dataloader, target_pairs))

    #REPLACE TEST PAIRS
    test_bleu = evaluateBleu(global_encoder_model, global_decoder_model, input_lang, output_lang, test_pairs, n=min(100,len(test_pairs)))

    return test_bleu

Plotting results
================

Plotting is done with matplotlib, using the array of loss values
`plot_losses` saved while training.


In [106]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder\'s predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder\'s
attention outputs for display later.


In [107]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:


In [108]:
def evaluateMultipleBleu(encoders, decoders, input_output_langs, n=10):
    bleu_sum = 0
    bleu_test_sum = 0
    lang_length = len(encoders)
    bleus = []
    test_bleus = []
    for i in range(lang_length):
        bleu = evaluateBleu(encoders[i], decoders[i], input_output_langs[i][0], input_output_langs[i][1], input_output_langs[i][2], n=n)
        bleu_sum += bleu
        bleus.append(bleu)
    for i in range(lang_length):
        test_bleu = evaluateBleu(encoders[i], decoders[i], input_output_langs[i][0], input_output_langs[i][1], input_output_langs[i][3], n=n)
        bleu_test_sum += test_bleu
        test_bleus.append(test_bleu)
    return (bleu_sum / lang_length), (bleu_test_sum / lang_length), bleus, test_bleus
        

# Federated Learning

In [112]:
import copy
from tqdm import tqdm
import csv

def training(encoders, decoders, input_output_lang, rounds, lr, ds, C, K, E, filename=None, batch_size=None, hidden_size=None, cifar_data_test = None,
             test_batch_size = None, classes_test = None, sch_flag = None, weighting=False, lexical_weights=None):
    """
    Function implements the Federated Averaging Algorithm from the FedAvg paper.
    Specifically, this function is used for the server side training and weight update

    Params:
      - model:           PyTorch model to train
      - rounds:          Number of communication rounds for the client update
      - batch_size:      Batch size for client update training
      - lr:              Learning rate used for client update training
      - ds:              Dataset used for training
      - data_dict:       Type of data partition used for training (IID or non-IID)
      - C:               Fraction of clients randomly chosen to perform computation on each round
      - K:               Total number of clients
      - E:               Number of training passes each client makes over its local dataset per round
      - tb_writer_name:  Directory name to save the tensorboard logs
    Returns:
      - model:           Trained model on the server
    """
    input_lang_target, output_lang_target, target_dataloader_target, pairs_target = get_dataloader(batch_size, language='cat')
    langs = ['bem', 'kin', 'swa']

    # global model weights
    global_encoder_weights = {key: value for key, value in encoders[0].state_dict().items() if 'embedding' not in key}
    global_decoder_weights = {key: value for key, value in decoders[0].state_dict().items() if 'embedding' not in key and 'out' not in key}

    # training loss
    train_loss = []
    test_loss = []
    test_accuracy = []
    best_bleu = 0
    best_test_bleu = 0
    # measure time
    start = time.time()

    lex_weighting = False
    if lexical_weights is not None:
        lex_weighting = True
    lexical_weights_encoder = lexical_weights
    lexical_weights_decoder = lexical_weights

    if filename is not None:
            with open(filename, 'a') as f:
                # create the csv writer
                writer = csv.writer(f)

                # write a row to the csv file
                writer.writerow(['Rounds', 'Learning Rate', 'Client Fraction', 'Client Number', 'Local Epochs', 'Batch Size', 'Hidden Size', 'Weighting', 'Lexical Weighting'])
                writer.writerow([rounds, lr, C, K, E, batch_size, hidden_size, weighting, lex_weighting])

    
    
    #hidden_size, encoder_models, decoder_models, input_lang, output_lang, target_dataloder, update_rate=5, encoder=True, dropout_p=0.1)

    for curr_round in range(1, rounds + 1):
        w_encoder, w_decoder, local_loss, num_pairs, round_lex_weight = [], [], [], [], []
        # Retrieve the number of clients participating in the current training
        m = max(int(C * K), 1)
        # Sample a subset of K clients according with the value defined before
        S_t = np.random.choice(range(K), m, replace=False)
        # For the selected clients start a local training
        for k in tqdm(S_t):
            # Compute a local update
            local_update = ClientUpdate(train_dataloader=ds[k], learning_rate=lr, epochs=E,
                                        sch_flag=sch_flag)
            # Update means retrieve the values of the network weights
            
            e_og = encoders[k].state_dict()
            d_og = decoders[k].state_dict()
            e_og.update(global_encoder_weights)
            d_og.update(global_decoder_weights)
            encoders[k].load_state_dict(e_og)
            decoders[k].load_state_dict(d_og)
            
            encoder_weights, decoder_weights, loss, num = local_update.train(encoders[k], decoders[k])

            w_encoder.append({key: value for key, value in copy.deepcopy(encoder_weights).items() if 'embedding' not in key})
            w_decoder.append({key: value for key, value in copy.deepcopy(decoder_weights).items() if 'embedding' not in key and 'out' not in key})
            local_loss.append(copy.deepcopy(loss))
            num_pairs.append(num)
            # if lex_weighting:
            #     round_lex_weight.append(lexical_weights[k])

        # lr = 0.999*lr
        # updating the global weights
        weights_avg_e = copy.deepcopy(w_encoder[0])

        weights_sum = sum(num_pairs)
        client_weights = [weight / weights_sum for weight in num_pairs]
        client_weights_encoder = lexical_weights_encoder
        client_weights_decoder = lexical_weights_decoder
        if weighting:
            client_weights_encoder = client_weights_decoder = client_weights
        if lex_weighting:
            # new_weights = lexical_weights # [100 - weight for weight in round_lex_weight]
            # new_weights = [100 - weight for weight in round_lex_weight]
            # sum_lex = sum(new_weights)
            # lexical_weights
            if weighting:
                client_weights = [(weight + lex_weight) / 2 for lex_weight, weight in zip(new_weights, client_weights)]
            else:
                client_weights_encoder = [0.25 + 0.25 * (weight / sum(lexical_weights_encoder)) for weight in lexical_weights_encoder]
                client_weights_decoder = [0.25 + 0.25 * (weight / sum(lexical_weights_decoder)) for weight in lexical_weights_decoder]
        
        print(f"Encoder:{client_weights_encoder}, Decoder:{client_weights_decoder}")

        for k in weights_avg_e.keys():
            if weighting or lex_weighting:
                weights_avg_e[k] *= client_weights_encoder[0] # client_weights[0]
                for i in range(1, len(w_encoder)):
                    weights_avg_e[k] += w_encoder[i][k] * client_weights_encoder[i] # client_weights[i]
                
            else:
                for i in range(1, len(w_encoder)):
                    weights_avg_e[k] += w_encoder[i][k]
                
                weights_avg_e[k] = torch.div(weights_avg_e[k], len(w_encoder))

        global_encoder_weights = weights_avg_e

        weights_avg_d = copy.deepcopy(w_decoder[0])

        for k in weights_avg_d.keys():
            if weighting or lex_weighting:
                weights_avg_d[k] *= client_weights_decoder[0] #client_weights[0]
                for i in range(1, len(w_decoder)):
                    weights_avg_d[k] += w_decoder[i][k] * client_weights_decoder[i] #client_weights[i]
                
            else:
                for i in range(1, len(w_decoder)):
                    weights_avg_d[k] += w_decoder[i][k]

                weights_avg_d[k] = torch.div(weights_avg_d[k], len(w_decoder))

        global_decoder_weights = weights_avg_d
        

        # if curr_round == 200:
        #     lr = lr / 2
        #     E = E - 1

        # if curr_round == 300:
        #     lr = lr / 2
        #     E = E - 2

        # if curr_round == 400:
        #     lr = lr / 5
        #     E = E - 3

        # move the updated weights to our model state dict
        # encoder.load_state_dict(global_encoder_weights)
        # decoder.load_state_dict(global_decoder_weights)

        # loss
        loss_avg = sum(local_loss) / len(local_loss)
        # print('Round: {}... \tAverage Loss: {}'.format(curr_round, round(loss_avg, 3)), lr)
        train_loss.append(loss_avg)

        # t_accuracy, t_loss = testing(model, cifar_data_test, test_batch_size, criterion, num_classes, classes_test)
        # test_accuracy.append(t_accuracy)
        # test_loss.append(t_loss)

        # if best_accuracy < t_accuracy:
        #     best_accuracy = t_accuracy
        # # torch.save(model.state_dict(), plt_title)
        # print(curr_round, loss_avg, t_loss, test_accuracy[0], best_accuracy)
        # # print('best_accuracy:', best_accuracy, '---Round:', curr_round, '---lr', lr, '----localEpocs--', E)
        bleu, test_bleu, bleu_list, test_bleu_list = evaluateMultipleBleu(encoders, decoders, input_output_lang, n=200)

        if best_bleu < bleu:
            best_bleu = bleu
        if best_test_bleu < test_bleu:
            best_test_bleu = test_bleu

        if filename is not None:
            with open(filename, 'a') as f:
                # create the csv writer
                writer = csv.writer(f)

                # write a row to the csv file
                writer.writerow([curr_round, loss_avg, bleu, best_bleu, test_bleu, best_test_bleu, str(bleu_list), str(test_bleu_list)])
        print(f"Round {curr_round} >> Loss: {loss_avg}, BLEU:{bleu}, TEST:{test_bleu}, LISTS:({str(bleu_list)}/{str(test_bleu_list)})")

        #Training Meta Model
        if curr_round % 5 == 0 and curr_round >= 25:

            # global_encoder_model, global_decoder_model, test_pairs = personalize('kir', 20, save=False, encoder_weights=global_encoder_weights, decoder_weights=global_decoder_weights, sample=None)
            objective_full = partial(objective, encoder_models = encoders, decoder_models = decoders, input_lang=input_lang_target, output_lang=output_lang_target, target_dataloader=target_dataloader_target, target_pairs=pairs_target, test_pairs=test_pairs, sources = langs, device = device)

            study = optuna.create_study(study_name="optimizing weights", direction="maximize", storage=f"sqlite:///studies/{filename[:-4]}.db", load_if_exists=True)
            study.optimize(objective_full, n_trials=10)

            # print(f"ENCODERS: {proportions_encoder}, {proportions_decoder}")
            print(f"PARAMS: {study.best_params}, VALUE: {study.best_value}")
            lexical_weights_encoder = []
            lexical_weights_decoder = []
            for i, (k,v) in enumerate(study.best_params.items()):
                if i % 2 == 0:
                    lexical_weights_encoder.append(v)
                else:
                    lexical_weights_decoder.append(v)
            lexical_weights_encoder.append(100 - sum(lexical_weights_encoder))
            lexical_weights_decoder.append(100 - sum(lexical_weights_decoder))
            

    end = time.time()
    print("Training Done!")
    print("Total time taken to Train: {}".format(end - start))

    return global_encoder_weights, global_decoder_weights

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we\'ll get
some reasonable results.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>If you run this notebook you can train, interrupt the kernel,evaluate, and continue training later. Comment out the lines where theencoder and decoder are initialized and run <code>trainIters</code> again.</p>
</div>


In [113]:
hidden_size = 128
batch_size = 32
# input_lang, output_lang, train_dataloader = get_dataloader(batch_size, language='spa')

In [114]:
MAX_LENGTH = 10
data_dict = {}
encoders = {}
decoders = {}
lexical_weight_dict = {}
input_output_lang = {}
langs = ['bem', 'kin', 'swa']
langs = sorted(langs)
weights = [42, 6, 43] # This needs to line up with the sorted langs
K = len(langs)
for i in range(K):
    input_lang, output_lang, train_dataloader, pair = get_dataloader(batch_size, language=langs[i])
    train_dataloader, test_dataloader, split = split_dataloader(train_dataloader)
    train_idx = get_pair_index(train_dataloader)
    test_idx = get_pair_index(test_dataloader)

    train_pairs = [pair[i] for i in train_idx]
    test_pairs = [pair[i] for i in test_idx]

    data_dict[i] = train_dataloader
    encoders[i] = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoders[i] = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
    input_output_lang[i] = (input_lang, output_lang, train_pairs, test_pairs)
    lexical_weight_dict[i] = weights[i]

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Reading lines...
Read 55667 sentence pairs
Trimmed to 19616 sentence pairs
Counting words...
Counted words:
kin 36940
eng 11235
Reading lines...
Read 272543 sentence pairs
Trimmed to 223701 sentence pairs
Counting words...
Counted words:
swa 37885
eng 27855


In [115]:
# st = {key: value for key, value in encoders[0].state_dict().items() if 'embedding' not in key}
# st2 = {key: value for key, value in decoders[0].state_dict().items() if 'embedding' not in key and 'out' not in key}
# encoders[1].state_dict().update(st)
# decoders[1].state_dict().update(st2)
# new = (encoders[0].state_dict())
# new.update(global_encoder_weights)
# encoders[0].load_state_dict(new)

In [116]:
import os
# train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

filename=None
save = True
if save:
    num = 1
    filename = f"GLOBAL|{'_'.join(langs)}||{num}.csv"
    while os.path.isfile(filename):
        print('Name is taken...trying again...')
        num += 1
        filename = f"GLOBAL|{'_'.join(langs)}||{num}.csv"

In [117]:
meta_encoder_weights, meta_decoder_weights = training(encoders, decoders, input_output_lang, 50,
                                                    lr=0.001, ds=data_dict, C=1.0, K=K, E=1,
                                                    filename=filename, batch_size=batch_size, hidden_size=hidden_size,
                                                    weighting=False, lexical_weights=[33, 33, 33])

Reading lines...
Read 1375 sentence pairs
Trimmed to 1131 sentence pairs
Counting words...
Counted words:
cat 1496
eng 1220


100%|██████████| 3/3 [02:03<00:00, 41.22s/it] 


Encoder:[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], Decoder:[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Round 1 >> Loss: 3.0863123060828896, BLEU:0.010310430079698563, TEST:0.00961930180589358, LISTS:([0.030931290239095688, 0.0, 0.0]/[0.02885790541768074, 0.0, 0.0])


100%|██████████| 3/3 [02:01<00:00, 40.33s/it]


Encoder:[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], Decoder:[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Round 2 >> Loss: 2.6472201207936354, BLEU:0.02047449102004369, TEST:0.024614808149635792, LISTS:([0.06142347306013107, 0.0, 0.0]/[0.06226855143904686, 0.0, 0.011575873009860516])


...

Encoder:[0.3625, 0.325, 0.3125], Decoder:[0.4, 0.2725, 0.3275]


[I 2024-09-10 11:04:10,817] Using an existing study with name 'optimizing weights' instead of creating a new one.


Round 50 >> Loss: 1.342395346700627, BLEU:0.1142835405965646, TEST:0.048968988781174026, LISTS:([0.22068636119365692, 0.07950806617736816, 0.04265619441866875]/[0.13074980676174164, 0.0, 0.016157159581780434])


[I 2024-09-10 11:04:31,339] Trial 50 finished with value: 0.022671451792120934 and parameters: {'bem_encoder': 19, 'bem_decoder': 59, 'kin_encoder': 62, 'kin_decoder': 17}. Best is trial 15 with value: 0.03435332328081131.
[I 2024-09-10 11:04:50,985] Trial 51 finished with value: 0.016834765034070165 and parameters: {'bem_encoder': 28, 'bem_decoder': 20, 'kin_encoder': 54, 'kin_decoder': 75}. Best is trial 15 with value: 0.03435332328081131.
[I 2024-09-10 11:05:11,131] Trial 52 finished with value: 0.015442506410181522 and parameters: {'bem_encoder': 47, 'bem_decoder': 52, 'kin_encoder': 47, 'kin_decoder': 29}. Best is trial 15 with value: 0.03435332328081131.
[I 2024-09-10 11:05:31,256] Trial 53 finished with value: 0.01124036405235529 and parameters: {'bem_encoder': 62, 'bem_decoder': 4, 'kin_encoder': 25, 'kin_decoder': 84}. Best is trial 15 with value: 0.03435332328081131.
[I 2024-09-10 11:05:51,207] Trial 54 finished with value: 0.0 and parameters: {'bem_encoder': 3, 'bem_decoder'

PARAMS: {'bem_encoder': 45, 'bem_decoder': 60, 'kin_encoder': 30, 'kin_decoder': 9}, VALUE: 0.03435332328081131
Training Done!
Total time taken to Train: 7554.2524263858795


In [ ]:
bleu_test_sum = 0
lang_length = len(encoders)
test_bleus = []

for i in tqdm(range(lang_length)):
    test_bleu = evaluateBleu(encoders[i], decoders[i], input_output_lang[i][0], input_output_lang[i][1], input_output_lang[i][3], n=500)
    bleu_test_sum += test_bleu
    test_bleus.append(test_bleu)

print(bleu_test_sum)
print(test_bleus)
test_bleu = bleu_test_sum / lang_length

if filename is not None:
    with open(filename, 'a') as f:
        # create the csv writer
        writer = csv.writer(f)

        # write a row to the csv file
        writer.writerow([test_bleu, str(test_bleus)])

100%|██████████| 2/2 [00:02<00:00,  1.18s/it]

0.1678844355046749
[0.12518097460269928, 0.04270346090197563]


In [ ]:
input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language='kir')

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

e_og = encoder.state_dict()
d_og = decoder.state_dict()
e_og.update(meta_encoder_weights)
d_og.update(meta_decoder_weights)
encoder.load_state_dict(e_og)
decoder.load_state_dict(d_og)

evaluateBleu(encoder, decoder, input_lang, output_lang, pairs, n=500)

Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566


0.0

In [ ]:
filename=None
save = True
if save:
    num = 1
    filename = f"FT|{'_'.join(langs)}||{num}.csv"
    while os.path.isfile(filename):
        print('Name is taken...trying again...')
        num += 1
        filename = f"FT|{'_'.join(langs)}||{num}.csv"

In [ ]:
ft_meta_encoder_weights, ft_meta_decoder_weights = training(encoders, decoders, input_output_lang, 10,
                                                    lr=0.001, ds=data_dict, C=1.0, K=K, E=1,
                                                    filename=filename, batch_size=batch_size, hidden_size=hidden_size,
                                                    weighting=False, lexical_weights=None)

100%|██████████| 2/2 [00:06<00:00,  3.04s/it]


[tensor(0.2472), tensor(0.7528)]
Round 1 >> Loss: 0.5035429736241385, BLEU:0.5134813487529755, TEST:0.07721402123570442, LISTS:([0.627179741859436, 0.3997829556465149]/[0.10407000035047531, 0.05035804212093353])


100%|██████████| 2/2 [00:06<00:00,  3.09s/it]


[tensor(0.2472), tensor(0.7528)]
Round 2 >> Loss: 0.493470582923376, BLEU:0.522781103849411, TEST:0.07091579400002956, LISTS:([0.6328366994857788, 0.4127255082130432]/[0.09516783803701401, 0.04666374996304512])


100%|██████████| 2/2 [00:06<00:00,  3.25s/it]


...

[tensor(0.7528), tensor(0.2472)]
Round 8 >> Loss: 0.4792762441519445, BLEU:0.5265385210514069, TEST:0.08649399876594543, LISTS:([0.6149012446403503, 0.4381757974624634]/[0.12205451726913452, 0.05093348026275635])


100%|██████████| 2/2 [00:06<00:00,  3.20s/it]


[tensor(0.2472), tensor(0.7528)]
Round 9 >> Loss: 0.4770499549143542, BLEU:0.5133463740348816, TEST:0.10136162303388119, LISTS:([0.6171251535415649, 0.40956759452819824]/[0.1499672383069992, 0.05275600776076317])


100%|██████████| 2/2 [00:06<00:00,  3.26s/it]


[tensor(0.2472), tensor(0.7528)]
Round 10 >> Loss: 0.475001929802373, BLEU:0.5240001082420349, TEST:0.08551276475191116, LISTS:([0.6289087533950806, 0.41909146308898926]/[0.1264883577823639, 0.044537171721458435])
Training Done!
Total time taken to Train: 83.34817051887512


In [ ]:
bleu_test_sum = 0
lang_length = len(encoders)
test_bleus = []

for i in tqdm(range(lang_length)):
    test_bleu = evaluateBleu(encoders[i], decoders[i], input_output_lang[i][0], input_output_lang[i][1], input_output_lang[i][3], n=500)
    bleu_test_sum += test_bleu
    test_bleus.append(test_bleu)
test_bleu = bleu_test_sum / lang_length

if filename is not None:
    with open(filename, 'a') as f:
        # create the csv writer
        writer = csv.writer(f)

        # write a row to the csv file
        writer.writerow([test_bleu, str(test_bleus)])

100%|██████████| 2/2 [00:02<00:00,  1.22s/it]


In [359]:
input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language='kir_test')

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

e_og = encoder.state_dict()
d_og = decoder.state_dict()
e_og.update(meta_encoder_weights)
d_og.update(meta_decoder_weights)
encoder.load_state_dict(e_og)
decoder.load_state_dict(d_og)

evaluateBleu(encoder, decoder, input_lang, output_lang, pairs, n=50)

Reading lines...
Read 7376 sentence pairs
Trimmed to 215 sentence pairs
Counting words...
Counted words:
kir_test 677
eng 575


0.0

In [360]:
import torch
from torch.utils.data import DataLoader, Subset

def limited_data_loader(original_dataloader, num_samples, random=True):
    # Get the original dataset from the DataLoader
    dataset = original_dataloader.dataset
    
    # Ensure the original dataset is larger than the requested number of samples
    assert len(dataset) >= num_samples, "The original dataset has fewer samples than requested"
    
    # Create a Subset of the original dataset limited to the first num_samples elements
    if random:
        indices = np.random.choice(len(dataset), num_samples, replace=False)
    else:
        indices = range(num_samples)

    subset = Subset(dataset, indices)
    
    # Create a new DataLoader from this subset with the same parameters as the original DataLoader
    new_dataloader = DataLoader(subset, batch_size=original_dataloader.batch_size, shuffle=False, num_workers=original_dataloader.num_workers)
    
    return new_dataloader


In [118]:
MAX_LENGTH = 11
input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language='kir')

Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292


In [119]:
# global_encoder_model, global_decoder_model, test_pairs = personalize('kir', 20, save=False, encoder_weights=global_encoder_weights, decoder_weights=global_decoder_weights, sample=None)
langs = ['bem', 'kin', 'swa']
objective_full = partial(objective, encoder_models = encoders, decoder_models = decoders, input_lang=input_lang, output_lang=output_lang, target_dataloader=train_dataloader, target_pairs=pairs, test_pairs=test_pairs, sources = langs, device = device)

study = optuna.create_study(study_name="optimizing weights", direction="maximize")
study.optimize(objective_full, n_trials=10)


[I 2024-09-10 11:07:33,893] A new study created in memory with name: optimizing weights
[I 2024-09-10 11:07:56,649] Trial 0 finished with value: 0.10102540999650955 and parameters: {'bem_encoder': 77, 'bem_decoder': 60, 'kin_encoder': 4, 'kin_decoder': 17}. Best is trial 0 with value: 0.10102540999650955.
[I 2024-09-10 11:08:19,709] Trial 1 finished with value: 0.11601746082305908 and parameters: {'bem_encoder': 62, 'bem_decoder': 11, 'kin_encoder': 35, 'kin_decoder': 9}. Best is trial 1 with value: 0.11601746082305908.
[I 2024-09-10 11:08:42,861] Trial 2 finished with value: 0.0590237134485107 and parameters: {'bem_encoder': 58, 'bem_decoder': 40, 'kin_encoder': 25, 'kin_decoder': 51}. Best is trial 1 with value: 0.11601746082305908.
[I 2024-09-10 11:09:05,929] Trial 3 finished with value: 0.018527010455727577 and parameters: {'bem_encoder': 97, 'bem_decoder': 24, 'kin_encoder': 2, 'kin_decoder': 51}. Best is trial 1 with value: 0.11601746082305908.
[I 2024-09-10 11:09:28,875] Trial 4

In [120]:
lexical_weights_encoder = []
lexical_weights_decoder = []
for i, (k,v) in enumerate(study.best_params.items()):
    if i % 2 == 0:
        lexical_weights_encoder.append(v)
    else:
        lexical_weights_decoder.append(v)
lexical_weights_encoder.append(100 - sum(lexical_weights_encoder))
lexical_weights_decoder.append(100 - sum(lexical_weights_decoder))

In [121]:
lexical_weights_encoder

[62, 35, 3]

In [122]:
encoder_weights = []
decoder_weights = []

for i in range(len(encoders)):
    encoder_weights.append({key: value for key, value in copy.deepcopy(encoders[i].state_dict()).items() if 'embedding' not in key})
    decoder_weights.append({key: value for key, value in copy.deepcopy(decoders[i].state_dict()).items() if 'embedding' not in key and 'out' not in key})
    
weights_avg_e = copy.deepcopy(encoder_weights[0])
weights_avg_d = copy.deepcopy(decoder_weights[0])

# Make state_dict empty with zeroes
for k in weights_avg_e:
    weights_avg_e[k].zero_()
for k in weights_avg_d:
    weights_avg_d[k].zero_()
        
# Add and combine weights of all input encoders/decoders according to proportion
for k in weights_avg_e.keys():
    for i in range(0, len(encoder_weights)):
        weights_avg_e[k] += lexical_weights_encoder[i] * encoder_weights[i][k] / 100

for k in weights_avg_d.keys():
    for i in range(0, len(decoder_weights)):
        weights_avg_d[k] += lexical_weights_decoder[i] * decoder_weights[i][k] / 100

In [70]:
study.best_params

{'fra_encoder': 33, 'fra_decoder': 25, 'por_encoder': 6, 'por_decoder': 34}

# Kirundi FT with OPTUNA

In [123]:
for i in range(3):
    encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=weights_avg_e, decoder_weights=weights_avg_d, sample=None)

Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
Epoch 25 >> Loss: 2.9879167353312175, Bleu: 0.14671051502227783, Test Bleu: 0.10284779220819473, Best:(0.16236938536167145/0.1246170923113823)
Epoch 50 >> Loss: 1.0147038218577704, Bleu: 0.31839656829833984, Test Bleu: 0.08487792313098907, Best:(0.3409104645252228/0.1405019909143448)
Epoch 75 >> Loss: 0.3709558785160382, Bleu: 0.4701131582260132, Test Bleu: 0.11458820104598999, Best:(0.5027176141738892/0.1405019909143448)
Epoch 100 >> Loss: 0.15678025535742446, Bleu: 0.6153135299682617, Test Bleu: 0.13251979649066925, Best:(0.6153135299682617/0.14854486286640167)
Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
Name is taken...trying again...
Epoch 25 >> Loss: 3.014275083700815, Bleu: 0.14696092903614044, Test Bleu: 0.13328596949577332, Best:(0.16720911860466003/0.14374946644516345)
Epoch

In [79]:
for i in range(3):
    encoder, decoder, test_pairs = personalize('fra', 100, save=True, sample=None)
for i in range(3):
    encoder, decoder, test_pairs = personalize('fra', 100, save=True, sample=1000)
for i in range(3):
    encoder, decoder, test_pairs = personalize('fra', 100, save=True, sample=5000)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
Epoch 25 >> Loss: 0.6567474312490937, Bleu: 0.615767776966095, Test Bleu: 0.3703753352165222, Best:(0.6264730095863342/0.3703753352165222)
Epoch 50 >> Loss: 0.05710218162581657, Bleu: 0.6533545255661011, Test Bleu: 0.34980589151382446, Best:(0.6997743844985962/0.38026905059814453)
Epoch 75 >> Loss: 0.02792180598213045, Bleu: 0.6576850414276123, Test Bleu: 0.3566093444824219, Best:(0.7141660451889038/0.38026905059814453)
Epoch 100 >> Loss: 0.022004641284853386, Bleu: 0.689117968082428, Test Bleu: 0.3513050079345703, Best:(0.7197762131690979/0.3814513087272644)
Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
Name is taken...trying again...
Epoch 25 >> Loss: 0.7018103707915369, Bleu: 0.4885055422782898, Test Bleu: 0.29379013180732727, Best:(0.49004215002059937/0.3094254732131958)
Epoch 

In [84]:
MAX_LENGTH = 10
for i in range(3):
    encoder, decoder, test_pairs = personalize('ita', 100, save=True, sample=None)
for i in range(3):
    encoder, decoder, test_pairs = personalize('ita', 100, save=True, sample=1000)
for i in range(3):
    encoder, decoder, test_pairs = personalize('ita', 100, save=True, sample=5000)

Reading lines...
Read 377937 sentence pairs
Trimmed to 325358 sentence pairs
Counting words...
Counted words:
ita 24907
eng 12302


KeyboardInterrupt: 

In [85]:
best_encoder = torch.load("best_encoder_kir.pt")
best_decoder = torch.load("best_decoder_kir.pt")

In [98]:
# for i in range(3):
encoder, decoder, test_pairs = personalize('bem', 100, save=True, encoder_weights=best_encoder, decoder_weights=best_decoder, sample=None)

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Epoch 25 >> Loss: 1.611445161452721, Bleu: 0.24702291190624237, Test Bleu: 0.12452561408281326, Best:(0.24952933192253113/0.14712001383304596)
Epoch 50 >> Loss: 0.658476777704451, Bleu: 0.36093440651893616, Test Bleu: 0.13057486712932587, Best:(0.38203662633895874/0.14712001383304596)
Epoch 75 >> Loss: 0.3817440288159935, Bleu: 0.42666181921958923, Test Bleu: 0.0965866670012474, Best:(0.42666181921958923/0.14712001383304596)
Epoch 100 >> Loss: 0.2572573367092894, Bleu: 0.4393371045589447, Test Bleu: 0.1297740787267685, Best:(0.4913264811038971/0.14712001383304596)


In [ ]:
EncoderRNN()

In [99]:
# for i in range(3):
encoder, decoder, test_pairs = personalize('bem', 100, save=True, sample=None)

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Epoch 25 >> Loss: 1.4166050180287855, Bleu: 0.2806829512119293, Test Bleu: 0.13488025963306427, Best:(0.3074606657028198/0.15606586635112762)
Epoch 50 >> Loss: 0.5020497167100542, Bleu: 0.444235235452652, Test Bleu: 0.11652989685535431, Best:(0.4777381420135498/0.161696657538414)
Epoch 75 >> Loss: 0.2823998911391659, Bleu: 0.5419302582740784, Test Bleu: 0.1265139877796173, Best:(0.5717427134513855/0.161696657538414)
Epoch 100 >> Loss: 0.19100201555611288, Bleu: 0.5408017635345459, Test Bleu: 0.14139288663864136, Best:(0.6021119952201843/0.161696657538414)


In [42]:
for i in range(3):
    encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=meta_encoder_weights, decoder_weights=meta_decoder_weights, sample=500)

Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
no shuffle
Epoch 25 >> Loss: 3.1243952166239426, Bleu: 0.1632363498210907, Test Bleu: 0.1130372640724038, Best:(0.1632363498210907/0.1403054487689914)
Epoch 50 >> Loss: 0.9226075988610587, Bleu: 0.4353345036506653, Test Bleu: 0.10907139629125595, Best:(0.44289126992225647/0.15641036044110726)
Epoch 75 >> Loss: 0.2371241340438525, Bleu: 0.6254980564117432, Test Bleu: 0.14093628525733948, Best:(0.6254980564117432/0.15641036044110726)
Epoch 100 >> Loss: 0.08585304656624794, Bleu: 0.6536855101585388, Test Bleu: 0.11871686577796936, Best:(0.678561270236969/0.15641036044110726)
Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
no shuffle
Name is taken...trying again...
Epoch 25 >> Loss: 3.15413605372111, Bleu: 0.12506474554538727, Test Bleu: 0.09209311008453369, Best:(0.13598057627677917/0.12668

In [43]:
for i in range(3):
    encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=meta_encoder_weights, decoder_weights=meta_decoder_weights, sample=100)

Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
no shuffle
Epoch 25 >> Loss: 4.323321930567424, Bleu: 0.06776860123267346, Test Bleu: 0.0686566434443879, Best:(0.06776860123267346/0.07215484772240972)
Epoch 50 >> Loss: 2.0862010780970257, Bleu: 0.3378818656288064, Test Bleu: 0.10700542794640341, Best:(0.3378818656288064/0.12132781744003296)
Epoch 75 >> Loss: 0.8709251209100085, Bleu: 0.6422285437583923, Test Bleu: 0.11082799095117929, Best:(0.6530925035476685/0.1319272518157959)
Epoch 100 >> Loss: 0.3214492837587993, Bleu: 0.7473016381263733, Test Bleu: 0.134121373295784, Best:(0.7549594640731812/0.1357509046792984)
Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
no shuffle
Name is taken...trying again...
Epoch 25 >> Loss: 4.23095806757609, Bleu: 0.06138278473261085, Test Bleu: 0.0563452492251914, Best:(0.06138278473261085/0.064189017

In [364]:
for i in range(3):
    encoder, decoder, test_pairs = personalize('kir', 100, save=True, sample=500)

Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
1317
no shuffle
Dataloader Train: 500
Dataloader Test: 330
Train examples 480
Test examples 320
Epoch 25 >> Loss: 3.2559069484074907, Bleu: 0.11196913570165634, Test Bleu: 0.0842679888010025, Best:(0.12159789353609085/0.0842679888010025)
Epoch 50 >> Loss: 1.0975287888844805, Bleu: 0.30702269077301025, Test Bleu: 0.1287538856267929, Best:(0.31274381279945374/0.14282378554344177)
Epoch 75 >> Loss: 0.3202398195664088, Bleu: 0.6449364423751831, Test Bleu: 0.11722522974014282, Best:(0.679939329624176/0.14745186269283295)
Epoch 100 >> Loss: 0.1069859682818254, Bleu: 0.7730781435966492, Test Bleu: 0.14985962212085724, Best:(0.7752388715744019/0.1517745852470398)
Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
1317
no shuffle
Dataloader Train: 500
Dataloader Test: 330
Train examples 480
Test exa

In [365]:
for i in range(3):
    encoder, decoder, test_pairs = personalize('kir', 100, save=True, sample=None)

Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
1317
Dataloader Train: 987
Dataloader Test: 330
Train examples 960
Test examples 320
Epoch 25 >> Loss: 2.780988239924113, Bleu: 0.15870223939418793, Test Bleu: 0.12819230556488037, Best:(0.1653084009885788/0.16161267459392548)
Epoch 50 >> Loss: 0.7319664926131567, Bleu: 0.3859574496746063, Test Bleu: 0.15479032695293427, Best:(0.4040026068687439/0.16351626813411713)
Epoch 75 >> Loss: 0.18656213439504307, Bleu: 0.5384177565574646, Test Bleu: 0.15235279500484467, Best:(0.5671566724777222/0.17379246652126312)
Epoch 100 >> Loss: 0.06217837663491566, Bleu: 0.5931922793388367, Test Bleu: 0.15370775759220123, Best:(0.6825713515281677/0.17379246652126312)
Reading lines...
Read 29506 sentence pairs
Trimmed to 1317 sentence pairs
Counting words...
Counted words:
kir 3038
eng 2292
1317
Dataloader Train: 987
Dataloader Test: 330
Train examples 960
Test examples 320
Name is t

In [270]:
torch.save(weights_avg_e, "./best_encoder_kir.pt")

In [271]:
torch.save(weights_avg_d, "./best_decoder_kir.pt")

In [267]:
encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=weights_avg_e, decoder_weights=weights_avg_d, sample=100)

Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566
Epoch 25 >> Loss: 3.529470497369766, Bleu: 0.058492448180913925, Test Bleu: 0.060413893312215805, Best:(0.058492448180913925/0.06182243302464485)


KeyboardInterrupt: 

In [274]:
weights_avg_e_old = weights_avg_e
weights_avg_d_old = weights_avg_d

In [96]:
sum(p.numel() for p in encoders[1].parameters())

4827392

In [363]:
bleus = []
input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language='kir')
for i in [None, 10, 100]:
    encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=ft_meta_encoder_weights, decoder_weights=ft_meta_decoder_weights, sample=i)

Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566
Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566
Epoch 5 >> Loss: 4.411212799549102, Bleu: 0.12715553465016713, Test Bleu: 0.136670250764458, Best:(0.12715553465016713/0.136670250764458)
Epoch 10 >> Loss: 2.998807468414307, Bleu: 0.1468493789434433, Test Bleu: 0.1660253405570984, Best:(0.19023461639881134/0.1704300920983599)
Epoch 15 >> Loss: 2.4015121924877163, Bleu: 0.18599043786525726, Test Bleu: 0.17539939284324646, Best:(0.20620320737361908/0.17539939284324646)
Epoch 20 >> Loss: 1.8836940324306486, Bleu: 0.22937878966331482, Test Bleu: 0.18200230598449707, Best:(0.22937878966331482/0.18452519178390503)
Epoch 25 >> Loss: 1.4516188883781433, Bleu: 0.30739349126815796, Test Bleu: 0.19906717538833618, Best:(0.30739349126815796/0.19906717538833618)
Epoch 30 >> Loss: 1.0872719359397887, Bleu

In [346]:
langs = ['bem', 'kin', 'lug']
parent_bleus = []

for lang in langs:
    input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language=lang)

    encoder, decoder, test_pairs = personalize(lang, 100, save=True, encoder_weights=ft_meta_encoder_weights, decoder_weights=ft_meta_decoder_weights, sample=None)

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Epoch 5 >> Loss: 2.7597879398700798, Bleu: 0.07422833889722824, Test Bleu: 0.08102406561374664, Best:(0.07751059532165527/0.08102406561374664)
Epoch 10 >> Loss: 1.9695946848273675, Bleu: 0.11479580402374268, Test Bleu: 0.07936546951532364, Best:(0.11752612143754959/0.11749737709760666)
Epoch 15 >> Loss: 1.623021117674552, Bleu: 0.11801541596651077, Test Bleu: 0.09571251273155212, Best:(0.1562468707561493/0.11749737709760666)
Epoch 20 >> Loss: 1.3789203233299065, Bleu: 0.1560976356267929, Test Bleu: 0.10654176026582718, Best:(0.1604466438293457/0.11936020851135254)
Epoch 25 >> Loss: 1.1935992406650238, Bleu: 0.18424397706985474, Test Bleu: 0.10948416590690613, Best:(0.2019069343805313/0.12215903401374817)
Epoch 30 >> Loss: 1.047508745078

In [347]:
langs = ['bem', 'kin', 'lug']

for lang in langs:
    input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language=lang)

    encoder, decoder, test_pairs = personalize(lang, 100, save=True, encoder_weights=ft_meta_encoder_weights, decoder_weights=ft_meta_decoder_weights, sample=100)

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Epoch 5 >> Loss: 7.435945653915406, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 10 >> Loss: 4.7951505303382875, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 15 >> Loss: 3.491755449771881, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 20 >> Loss: 3.0078428626060485, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 25 >> Loss: 2.67136572599411, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 30 >> Loss: 2.366993796825409, Bleu: 0.0, Test Bleu: 0.011165823635936115, Best:(0/0.011165823635936115)
Epoch 35 >> Loss: 2.0924447149038317, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0.01909972642142283)
Epoch 40 >> Loss: 1.837030079960823, Bleu: 0.013807805196919083, Test Bleu: 0.0, Best:(0.013807805196919083/0.01909972642142283)
Epoch 45 >> Loss: 1.6064924001693726,

In [321]:
langs = ['bem', 'kin', 'lug']

for lang in langs:
    input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language=lang)

    encoder, decoder, test_pairs = personalize(lang, 100, save=True, encoder_weights=None, decoder_weights=None, sample=None)

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Epoch 5 >> Loss: 2.533391289972388, Bleu: 0.11117631942033768, Test Bleu: 0.09661664813756943, Best:(0.1467665284872055/0.11440379917621613)
Epoch 10 >> Loss: 1.6053250909445687, Bleu: 0.1856374442577362, Test Bleu: 0.1318291872739792, Best:(0.18662051856517792/0.1629309505224228)
Epoch 15 >> Loss: 1.2140471017439896, Bleu: 0.2066066861152649, Test Bleu: 0.1401309370994568, Best:(0.21999995410442352/0.1629309505224228)
Epoch 20 >> Loss: 0.9653794986861092, Bleu: 0.234907329082489, Test Bleu: 0.16295795142650604, Best:(0.26048076152801514/0.16295795142650604)
Epoch 25 >> Loss: 0.7909483602573705, Bleu: 0.3547486364841461, Test Bleu: 0.14942236244678497, Best:(0.3547486364841461/0.16923066973686218)
Epoch 30 >> Loss: 0.6631389268113924, B

In [322]:
langs = ['bem', 'kin', 'lug']

for lang in langs:
    input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language=lang)

    encoder, decoder, test_pairs = personalize(lang, 100, save=True, encoder_weights=None, decoder_weights=None, sample=100)

Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Reading lines...
Read 82370 sentence pairs
Trimmed to 25719 sentence pairs
Counting words...
Counted words:
bem 19481
eng 5092
Epoch 5 >> Loss: 6.797128224372864, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 10 >> Loss: 4.100341749191284, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 15 >> Loss: 3.3996550679206847, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 20 >> Loss: 3.097805458307266, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 25 >> Loss: 2.8388034045696258, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 30 >> Loss: 2.5928813815116882, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 35 >> Loss: 2.3535727679729463, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 40 >> Loss: 2.1276715636253356, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 45 >> Loss: 1.9152377903461457, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 50 >> Loss: 1.7170710533857345, Bleu: 0.0, Test Bleu: 0.0, B

In [324]:
input_lang, output_lang, train_dataloader, pairs = get_dataloader(batch_size, language='kir')

encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=None, decoder_weights=None, sample=None)
encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=None, decoder_weights=None, sample=100)

Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566
Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566
Epoch 5 >> Loss: 4.294888372421265, Bleu: 0.0845878895901942, Test Bleu: 0.07503550499677658, Best:(0.08475242555141449/0.07521585375070572)
Epoch 10 >> Loss: 3.087857704162597, Bleu: 0.07980252057313919, Test Bleu: 0.0649627298116684, Best:(0.0967574268579483/0.07521585375070572)
Epoch 15 >> Loss: 2.4582161164283756, Bleu: 0.08963023126125336, Test Bleu: 0.07968392223119736, Best:(0.09816891700029373/0.07968392223119736)
Epoch 20 >> Loss: 1.9043814027309416, Bleu: 0.1049438938498497, Test Bleu: 0.08781197667121887, Best:(0.11620661616325378/0.08781197667121887)
Epoch 25 >> Loss: 1.4426589131355285, Bleu: 0.14083968102931976, Test Bleu: 0.08822551369667053, Best:(0.1463944911956787/0.0935228019952774)
Epoch 30 >> Loss: 1.0770999479293824, Ble

In [325]:
noweight_encoderft = ft_meta_encoder_weights
noweight_decoderft = ft_meta_decoder_weights
noweight_encoder = meta_encoder_weights
noweight_decoder = meta_decoder_weights

In [348]:
encoder, decoder, test_pairs = personalize('kir', 100, save=True, encoder_weights=None, decoder_weights=None, sample=10)

Reading lines...
Read 29506 sentence pairs
Trimmed to 872 sentence pairs
Counting words...
Counted words:
kir 1954
eng 1566
Epoch 5 >> Loss: 7.091353511810302, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 10 >> Loss: 6.328334522247315, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 15 >> Loss: 5.28613052368164, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 20 >> Loss: 4.29144926071167, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 25 >> Loss: 3.5186821937561037, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 30 >> Loss: 2.9056015014648438, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 35 >> Loss: 2.422715997695923, Bleu: 0.0, Test Bleu: 0.0, Best:(0/0)
Epoch 40 >> Loss: 2.0179399251937866, Bleu: 0.003873996203765273, Test Bleu: 0.0, Best:(0.00456723477691412/0)
Epoch 45 >> Loss: 1.657440185546875, Bleu: 0.00756849953904748, Test Bleu: 0.0, Best:(0.007926229387521744/0.004629823379218578)
Epoch 50 >> Loss: 1.349088931083679, Bleu: 0.0, Test Bleu: 0.0, Best:(0.015829095616936684/0.004629823379218578)

In [349]:
lexical_encoderft = ft_meta_encoder_weights
lexical_decoderft = ft_meta_decoder_weights
lexical_encoder = meta_encoder_weights
lexical_decoder = meta_decoder_weights